In [ ]:
import nibabel as nib
import numpy as np
import torch
import os
from monai.transforms import (
    EnsureChannelFirstd,
    LoadImaged,
    RandFlipd,
    RandRotate90d,
    SignalFillEmptyd,
    Compose,
    ConcatItemsd,
    ToTensord,
    RandShiftIntensityd,
    CropForegroundd,
    Orientationd,
    Spacingd,
    Transform,
    MapTransform,
    NormalizeIntensityd,
    LambdaD,
    RandGaussianNoised,
    RandGaussianSmoothd,
    RandCropByLabelClassesd,
)

ct_path = "/Users/keyi/Desktop/MA/AutoPet_Anatomy/FDG-PET-CT-Lesions/PETCT_0af7ffe12a/08-12-2005-NA-PET-CT Ganzkoerper  primaer mit KM-96698/CTres.nii.gz"  
save_dir = "output"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "ct_transform.nii.gz")

class ClipPercentiles(Transform):
    def __init__(self, lower_percentile:float, upper_percentile:float) -> None:
        super().__init__()
        self.lower_percentile = lower_percentile
        self.upper_percentile = upper_percentile

    def __call__(self, img:torch.Tensor):
        lower_threshold =   np.quantile(img,self.lower_percentile)
        upper_threshold = np.quantile(img,self.upper_percentile)
        return torch.clamp(img, min=lower_threshold, max=upper_threshold)

class ClipPercentaged(MapTransform):
    def __init__(self, keys, lower_percentile: float, upper_percentile: float) -> None:
        super().__init__(keys)
        self.clip_transform = ClipPercentiles(lower_percentile, upper_percentile)
    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            d[key] = self.clip_transform(d[key])
        return d
        
transforms = Compose([
    LoadImaged(keys=["image"]),
    EnsureChannelFirstd(keys=["image"]),
    Orientationd(keys=["image"], axcodes="RAS"),
    Spacingd(
        keys=["image"],
        pixdim=(2.04, 2.04, 3.0), 
        mode=("bilinear")
    ),
    ClipPercentaged(
            keys=["image"],
            lower_percentile=0.5,
            upper_percentile=0.995
        ),
    NormalizeIntensityd(
            keys=["image"],
            subtrahend=-79.55364831013831,
            divisor=158.5583080580807
        ),
    RandGaussianNoised(keys=["image"], std=0.1, prob=0.15),
        RandGaussianSmoothd(
            keys=["image"],
            sigma_x=(0.5, 1),
            sigma_y=(0.5, 1),
            sigma_z=(0.5, 1),
            prob=0.2,
        ),
        # random flip along the spatial axis (all axes)
        RandFlipd(keys=["image"], 
                  spatial_axis=[0,1,2], 
                  prob=0.05
                  ),
        RandRotate90d(keys=["image"], 
                      prob=0.05
                      ),
        RandShiftIntensityd(
            keys=["image"],
            offsets=0.15,
            prob=0.1
        ),
        SignalFillEmptyd(
            keys=["image"]
            ),
])


data = {"image": ct_path}
transformed_data = transforms(data)
ct_data = transformed_data["image"]

print(ct_data.shape)
if isinstance(ct_data, torch.Tensor):
    ct_data = ct_data.cpu().numpy()



original_img = nib.load(ct_path)
affine = original_img.affine
ct_data = ct_data[0]
print(f"now ct_data.shape is {ct_data.shape}")
nib.save(nib.Nifti1Image(ct_data, affine), 'ct_transform_0af7ffe12a.nii.gz')



In [ ]:
import nibabel as nib
import numpy as np
import torch
import os
from monai.transforms import (
    EnsureChannelFirstd,
    LoadImaged,
    RandFlipd,
    RandRotate90d,
    SignalFillEmptyd,
    Compose,
    ConcatItemsd,
    ToTensord,
    RandShiftIntensityd,
    CropForegroundd,
    Orientationd,
    Spacingd,
    Transform,
    MapTransform,
    NormalizeIntensityd,
    LambdaD,
    RandGaussianNoised,
    RandGaussianSmoothd,
    RandCropByLabelClassesd,
    GaussianSmoothd,
    ScaleIntensityd,
)

ct_path = "/Users/keyi/Desktop/MA/AutoPet_Anatomy/FDG-PET-CT-Lesions/PETCT_0af7ffe12a/08-12-2005-NA-PET-CT Ganzkoerper  primaer mit KM-96698/CTres.nii.gz"  
save_dir = "output"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "ct_transform.nii.gz")

class ClipPercentiles(Transform):
    def __init__(self, lower_percentile:float, upper_percentile:float) -> None:
        super().__init__()
        self.lower_percentile = lower_percentile
        self.upper_percentile = upper_percentile

    def __call__(self, img:torch.Tensor):
        lower_threshold =   np.quantile(img,self.lower_percentile)
        upper_threshold = np.quantile(img,self.upper_percentile)
        return torch.clamp(img, min=lower_threshold, max=upper_threshold)

class ClipPercentaged(MapTransform):
    def __init__(self, keys, lower_percentile: float, upper_percentile: float) -> None:
        super().__init__(keys)
        self.clip_transform = ClipPercentiles(lower_percentile, upper_percentile)
    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            d[key] = self.clip_transform(d[key])
        return d
        
MONAI_BUNDLE_TRANSFORMS = Compose([
    LoadImaged(keys=["image"]),
    EnsureChannelFirstd(keys=["image"]),
    Orientationd(keys=["image"], axcodes="RAS"),
    Spacingd(
        keys=["image"],
        pixdim=(1.5,1.5,1.5), 
        mode=("bilinear")
    ),
    NormalizeIntensityd(
            keys=["image"],
            nonzero=True,
        ),
    GaussianSmoothd(
        keys=["image"],
        sigma=0.4
    ),
    ScaleIntensityd(
        keys=["image"],
        minv=-1.0,
        maxv=1.0
    ),
    
])


data = {"image": ct_path}
transformed_data = MONAI_BUNDLE_TRANSFORMS(data)
ct_data = transformed_data["image"]

print(ct_data.shape)
if isinstance(ct_data, torch.Tensor):
    ct_data = ct_data.cpu().numpy()



original_img = nib.load(ct_path)
affine = original_img.affine
ct_data = ct_data[0]
print(f"now ct_data.shape is {ct_data.shape}")
nib.save(nib.Nifti1Image(ct_data, affine), 'ct_monai_transform_0af7ffe12a.nii.gz')

